In [15]:
%load_ext autoreload
%autoreload 2

import os 
from pathlib import Path

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

def mkdir(path: Path) -> Path:
    path = Path(path)
    if path.suffix != '':
        path = path.parent
    if path.exists():
        print('path exists, leaving alone')
    else:
        path.mkdir(parents=True)
    return path

this_dir = Path('').parent
TMP = mkdir('./tmp/out')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%%capture out

import jax
from jax import numpy as jnp
from PIL import Image

from flax import jax_utils
from flax.training import common_utils, train_state, dynamic_scale
import optax

def flatten(x):
  return x.reshape(x.shape[0], -1)

def trs(b):
    return jnp.array(b.numpy())[None]

In [79]:
import paramiko
import sys
import subprocess
import wandb
from time import sleep
from functools import partial, reduce
from itertools import product
from simple_slurm import Slurm
import random
from typing import Any, Iterable
import re
from ast import literal_eval

def get_cartesian_product(*args):
    """ Cartesian product is the ordered set of all combinations of n sets """
    return list(product(*args))

def zip_in_n_chunks(arg: Iterable[Any], n: int) -> zip:   
    return zip(*([iter(arg)]*n))

def flat_list(lst_of_lst):
    return [lst for sublst in lst_of_lst for lst in sublst]

def gen_alphanum(n: int = 7, test=False):
    from string import ascii_lowercase, ascii_uppercase
    random.seed(test if test else None)
    numbers = ''.join([str(i) for i in range(10)])
    characters = ascii_uppercase + ascii_lowercase + numbers
    name = ''.join([random.choice(characters) for _ in range(n)])
    return name

def add_to_Path(path: Path, string: str | Path):
        return Path(str(path) + str(string))

def iterate_folder(folder: Path, iter_exp_dir):
    if iter_exp_dir and folder.exists():
        for i in range(100):
            _folder = add_to_Path(folder, f'-{i}')
            if not re.search(_folder.name, f'-[0-9]*'):
                folder = _folder
                break
        else:
            folder = add_to_Path(folder, f'-0')
    return folder

In [94]:
x = set()
x.update({'max'})
x.update({'can'})
x

{'can', 'max'}

In [ ]:
stdout = run_cmds(['git log -l'], cwd=_i.project_dir)
stdout.decode('utf-8').replace('\n', ' ').split(' ')[1]

In [100]:
class Sub:
    ignore_attr = ['parent','protected','dict','cmd']
    
    def __init__(_i, parent):
        super().__init__() 
        _i.parent = parent
        
        for k,v in cls_to_dict(_i, Sub.ignore_attr).items(): # dictfromcls pulls values (property agnostic),
            cls_v = getattr(_i.__class__, k)
            if not isinstance(cls_v, property): # check the dict if property
                setattr(_i, k, v)  # initialising subclasses

    @property
    def dict(_i,):
        d = cls_to_dict(_i, Sub.ignore_attr)
        for k,v in d.items():
            if issubclass(type(v), Sub):
                d[k] = cls_to_dict(v, Sub.ignore_attr)
        return d

def cls_to_dict(cls, ignore:list)->dict:
    return {k: getattr(cls, k) for k in dir(cls) if not (k.startswith('_') or k in ignore)}

def cmd_to_dict(cmd:str|list,ref:dict,_d={},delim:str=' --'):
    """
    fmt: [--flag, arg, --true_flag, --flag, arg1]
    # all flags double dash because of negative numbers duh """
    booleans = ['True', 'true', 't', 'False', 'false', 'f']
    
    cmd = ' '.join(cmd) if isinstance(cmd, list) else cmd
    cmd = [x.lstrip().lstrip('--').rstrip() for x in cmd.split(delim)]
    cmd = [x.split(' ', maxsplit=1) for x in cmd if ' ' in x]
    [x.append('True') for x in cmd if len(x) == 1]
    cmd = flat_list(cmd)
    cmd = iter([x.strip() for x in cmd])

    for k,v in zip(cmd, cmd):
        if v in booleans: 
            v=booleans.index(v)<3  # 0-2 True 3-5 False
        if k in ref:
            _d[k] = type(ref[k])(v)
        else:
            try:
                _d[k] = literal_eval(v)
            except:
                _d[k] = str(v)
            print(f'Guessing type: {k} as {type(v)}')
    return _d

def update_cls_with_dict(cls: Any, d:dict):
    cls_all = [Sub for Sub in cls.__dict__.values() if issubclass(type(v), Sub)]
    cls_all.extend(cls)
    n_remain = len(d)
    for k,v in d.items():
        for _cls_assign in cls_all:            
            if not hasattr(_cls_assign, k):
                continue
            else:
                if isinstance(cls.__class__.__dict__[k], property):
                    print('Tried to assign property, consider your life choices')
                    continue
                v = type(cls.__dict__)(v)
                setattr(_cls_assign, k, v)
                n_remain -= 1
    return n_remain

def cls_to_dict(cls, ignore:list)->dict:
    return {k: getattr(cls, k) for k in dir(cls) if not (k.startswith('_') or k in ignore)}

def flat_dict(d:dict,items:list=[]):
    for k,v in d.items():
        if isinstance(v, dict):
            items.extend(flat_dict(v).items(),items=items)
        else:
            items.append((k, v))
    return dict(items)  

def dict_to_wandb(d:dict,parent_key:str='',sep:str ='.',items:list=[])->dict:
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict): 
            items.extend(dict_to_wandb(v,new_key,items=items).items())
        else:
            if isinstance(v, Path):  v=str(v)
            items.append((new_key, v))
    return dict(items)

def count_gpu() -> int: # output = run_cmd('echo $CUDA_VISIBLE_DEVICES', cwd='.')
    return sum(c.isdigit() for c in os.environ.get('CUDA_VISIBLE_DEVICES'))

def run_cmds(cmd:str|list,cwd:str|Path=None,input_req:str=None,_out=[]):
    for cmd_1 in (cmd if isinstance(cmd, list) else [cmd]): 
        _out += [subprocess.run(
            [c.strip() for c in cmd_1.split(' ')],cwd=cwd, input=input_req, capture_output=True)]
        sleep(0.1)
    return _out

def run_cmds_server(server:str,user:str,cmd:str|list,cwd=str|Path,_out=[]):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())  # if not known host
    client.connect(hostname=server, username=user)
    client.exec_command(f'cd {cwd}')
    with client as _r:
        for cmd_1 in (cmd if isinstance(cmd, list) else [cmd]):
            _out += [_r.exec_command(f'{cmd_1}')] # in, out, err
            sleep(0.1)
    return _out

class Pyfig(Sub):

    project:            str     = 'hwat'
    project_dir:        Path    = Path().absolute()
    n_device:           int     = property(lambda _i: _i.count_gpu())
    
    seed:               int     = 808017424         # grr

    exp_name:           str     = 'junk'
    run_path:           Path    = property(lambda _i: _i.project_dir / 'run.py')
    data_dir:          Path     = Path().home() / 'data'
    
    half_precision = True
    dtype:              str     = 'f32'
    n_step:             int     = 1000

    n_layer:            int     = 3
    af:                 str     = 'tanh'    # activation function
    
    class data(Sub):
        dataset = 'fashion_mnist'
        b_size = 16
        cache = False
        image_size = 28
        channels = 1

    class model(Sub):
        dim = 64
        dim_mults = (1, 2, 4)

    class opt(Sub):
        optimizer = 'Adam'
        beta1 = 0.9
        beta2 = 0.99
        eps = 1e-8
        lr = 0.001
        loss = 'l1'  # change this to loss table load? 

    class sweep(Sub):
        method = 'random'
        name = 'sweep'
        metrics = dict(
            goal = 'minimize',
            name = 'validation_loss',
        )
        parameters = dict(
            batch_size = {'values': [16, 32, 64]},
            epoch = {'values': [5, 10, 15]},
            lr = {'max': 0.1, 'min': 0.0001},
        )
        n_sweep = run_cap = reduce(
            lambda i0,i1: i0*i1, [len(v['values']) for k,v in parameters.items() if 'values' in v])+1
        sweep_id = ''

    class wandb(Sub):
        job_type:           str     = 'training'
        entity:             str     = 'xmax1'

    log_sample_step:    int     = 5
    log_metric_step:    int     = 5
    log_state_step:     int     = 10         # wandb entity
    n_epoch:            int     = 20

    class slurm(Sub):
        output          = TMP/'o-%j.out'
        error           = TMP/'e-%j.err'
        mail_type       = 'FAIL'
        partition       ='sm3090'
        nodes           = 1                # n_node
        ntasks          = 8                # n_cpu
        cpus_per_task   = 1     
        time            = '0-12:00:00'     # D-HH:MM:SS
        gres            = 'gpu:RTX3090:1'
        job_name        = property(lambda sub_i: sub_i.parent.exp_name)  # this does not call the instance it is in
        sbatch          = property(lambda sub_i: f""" 
            module purge 
            source ~/.bashrc 
            module load GCC 
            module load CUDA/11.4.1 
            module load cuDNN/8.2.2.26-CUDA-11.4.1 
            conda activate {sub_i.parent.env} 
            export MKL_NUM_THREADS=1 
            export NUMEXPR_NUM_THREADS=1 
            export OMP_NUM_THREADS=1 
            export OPENBLAS_NUM_THREADS=1
            pwd
            nvidia-smi
            mv_cmd = f'mv {TMP}/o-$SLURM_JOB_ID.out {TMP}/e-$SLURM_JOB_ID.err $out_dir' 
        """)

    exp_id:             str     = gen_alphanum(n=7)
    
    iter_exp_dir:       bool    = True
    server_project_dir: Path    = property(lambda _i: _i.project_dir)
    project_exp_dir:    Path    = property(lambda _i: _i.project_dir / 'exp')
    project_cfg_dir:    Path    = property(lambda _i: _i.project_dir / 'cfg')
    exp_path:           Path    = property(lambda _i: iterate_folder(_i.project_exp_dir / _i.exp_name, _i.iter_exp_dir) / _i.exp_id)

    server:             str     = 'svol.fysik.dtu.dk'   # SERVER
    user:               str     = 'amawi'     # SERVER
    entity:             str     = 'xmax1'       # WANDB entity
    git_remote:         str     = 'origin'      
    git_branch:         str     = 'main'        
    env:                str     = 'dex'            # CONDA ENV
    commit_id:          str     = property(lambda _i: _i.get_commit_id())
    
    _cluster_state:     int     = -1

    sys_arg: list = sys.argv[1:]

    def __init__(_i, args: dict={}, cap=40) -> None:
        
        super().__init__(None)
        
        [setattr(_i, k, v(_i)) for k,v in Pyfig.__dict__.items() if isinstance(v, type)]

        wandb.init(
            job_type=_i.wandb.job_type,
            entity=_i.wandb.entity,
            project=_i.project,
            config=dict_to_wandb(_i.dict),
            settings=wandb.Settings(start_method='fork'),  # idk why this is an issue
            dir=_i.exp_path,
        )

        update_cls_with_dict(args | cmd_to_dict(sys.argv[1:]))
        
        if _i._cluster_state > 0:
            n_job_running = _i.run_cmds([f'squeue -u {_i.parent.user} -h -t pending,running -r | wc -l'])
            if n_job_running > cap:
                exit(f'There are {n_job_running} on the cluster cap is {cap}')
            
            _slurm = Slurm(**_i.slurm)

            n_run, _i.clusterSubmit_state = _i.clusterSubmit_state, 0
            exit()
            for _ in range(_i._cluster_state):
                _slurm.sbatch(_i.slurm.sbatch 
                + f'out_dir={(mkdir(_i.exp_path/"out"))} {_i.cmd} | tee $out_dir/py.out date "+%B %V %T.%3N" ')

    @property
    def cmd(_i,):
        d = flat_dict(_i.dict)
        return ' '.join([f' --{k}  {str(v)} ' for k,v in d.items()])

    def clusterSubmit(_i, sweep=False, msg=None, cap=40):
        msg = msg or _i.exp_id
        _i._cluster_state *= -1 # init is -1
        if _i._cluster_state > 0:
            if sweep:
                _i.sweep_id = wandb.sweep(
                    env     = f'conda activate {_i.env};',
                    sweep   = _i.sweep, 
                    program = _i.run_path,
                    project = _i.project,
                    name    = _i.exp_name,
                    run_cap = _i.sweep.n_sweep
                )
                _i._cluster_state *= _i.sweep.n_sweep
            
            local_out = run_cmds(['git add .', f'git commit -m {msg}', 'git push'], cwd=_i.project_dir)
            server_out = run_cmds_server(_i.server, _i.user, f'python -u {_i.run_path} ' +_i.cmd, cwd=_i.server_project_dir)

    @property
    def commit_id(_i,)->str:
        stdout = run_cmds(['git log -l'], cwd=_i.project_dir)
        return stdout.decode('utf-8').replace('\n', ' ').split(' ')[1]
            
c = Pyfig()

AttributeError: 'list' object has no attribute 'decode'

In [109]:
def run_cmds(cmd:str|list,cwd:str|Path=None,input_req:str=None,_out=[]):

    for cmd_1 in (cmd if isinstance(cmd, list) else [cmd]): 
        print(cmd_1)
        cmd_1 = [c.strip() for c in cmd_1.split(' ')]
        print(cmd_1)
        _out += [subprocess.run(
            cmd_1,cwd=cwd, input=input_req, capture_output=True)]
        sleep(0.1)
    return _out

stdout = run_cmds(['git log -l'], cwd='/home/amawi/projects/hwat')
# stdout.decode('utf-8').replace('\n', ' ').split(' ')[1]
stdout

git log -l
['git', 'log', '-l']


[CompletedProcess(args=['git', 'log', '-l'], returncode=128, stdout=b'', stderr=b"fatal: Option 'l' requires a value\n")]

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: ', device)

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

def get_fashion_loader(b_size, data_dir, mean=None, std=None, n_workers=4):
    
    data_tr = datasets.FashionMNIST(
        root=data_dir, 
        download=True, 
        train=True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(28),
            transforms.CenterCrop(28),
            transforms.Normalize((mean), (std)),
        ]),
        target_transform = transforms.Compose([
        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
        ])
    )

    loader_tr = DataLoader(
        data_tr, 
        batch_size=b_size, 
        shuffle=True, 
        num_workers=n_workers,
    )

    data_test = datasets.FashionMNIST(
        root=data_dir, 
        download=True, 
        train=False,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Normalize((mean), (std)),
        ]),
        target_transform = transforms.Compose([
        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
    ])
    )

    loader_test = DataLoader(
        data_test, 
        batch_size=b_size, 
        shuffle=True, 
        num_workers=n_workers,
    )
    return loader_tr, loader_test

In [ ]:
rng = jax.random.PRNGKey(c.seed)
rng, subrng = jax.random.split(rng)

from typing import Any

class TrainState(train_state.TrainState):
    dynamic_scale: Optional[dynamic_scale_lib.DynamicScale] = None
    params_ema: Any = None

def create_train_state(rng, dynamic_scale=None):
    
    model = unet.Unet(
        dim = c.model.dim, 
        out_dim =  c.data.channels,
        dim_mults = c.model.dim_mults
    )
    input_dim = c.data.channels * 2 if c.ddpm.self_condition else c.data.channels
    input_shape = (1, c.data.image_size, c.data.image_size, input_dim)

    def initialized():
        @jax.jit
        def init(*args):
            return model.init(*args)
        variables = init(
            {'params': rng}, 
            jnp.ones(input_shape, model.dtype), # x noisy image
            jnp.ones(input_shape[:1], model.dtype) # t
            )
        return variables['params']
    
    tx = optax.adam(learning_rate=c.opt.lr , b1=c.opt.beta1, b2 = c.opt.beta2, eps=c.opt.eps)

    params = initialized()
    state = TrainState.create(
        apply_fn=model.apply, 
        params=params,
        params_ema=params,
        tx=tx, 
        dynamic_scale=dynamic_scale
    )

    return state
    
state = create_train_state(rng)
state = jax_utils.replicate(state)

In [ ]:
def create_ema_decay_schedule(update_after_step, inv_gamma, power, min_value, beta, **kw):

    def ema_decay_schedule(step):
        count = jnp.clip(step -  update_after_step - 1, a_min = 0.)
        value = 1 - (1 + count /  inv_gamma) ** -  power 
        ema_rate = jnp.clip(value, a_min =  min_value, a_max =  beta)
        return ema_rate

    return ema_decay_schedule
    
ema_decay_fn = create_ema_decay_schedule(**c.ema.dict)

def cosine_beta_schedule(timesteps):
    """Return cosine schedule 
    as proposed in https://arxiv.org/abs/2102.09672 """
    s=0.008
    max_beta=0.999
    ts = jnp.linspace(0, 1, timesteps + 1)
    alphas_bar = jnp.cos((ts+s)/(1+s)*jnp.pi/2) ** 2
    alphas_bar = alphas_bar/alphas_bar[0]
    betas = 1 - (alphas_bar[1:] / alphas_bar[:-1])
    return jnp.clip(betas, 0, max_beta)

class ddpm_param:
    timesteps = c.ddpm.timesteps
    p2_loss_weight_gamma = c.ddpm.p2_loss_weight_gamma
    p2_loss_weight_k = c.ddpm.p2_loss_weight_gamma

    betas = cosine_beta_schedule(timesteps)

    assert betas.shape == (timesteps,)
    alphas = 1. - betas
    alphas_bar = jnp.cumprod(alphas, axis=0)
    sqrt_alphas_bar = jnp.sqrt(alphas_bar)
    sqrt_1m_alphas_bar= jnp.sqrt(1. - alphas_bar)
    
    # calculate p2 reweighting
    p2_loss_weight=  (p2_loss_weight_k + alphas_bar / (1 - alphas_bar)) ** -p2_loss_weight_gamma

In [ ]:
def l2_loss(logit, target):
    return (logit - target)**2

def l1_loss(logit, target): 
    return jnp.abs(logit - target)

loss_table = dict(
    l1_loss = l1_loss,
    l2_loss = l2_loss
)

loss_fn = loss_table[c.opt.loss_fn]

In [ ]:
def noise_to_x0(noise, xt, batched_t, alphas_bar, sqrt_alphas_bar):
    assert batched_t.shape[0] == xt.shape[0] == noise.shape[0] # make sure all has batch dimension
    sqrt_alpha_bar = sqrt_alphas_bar[batched_t, None, None, None]
    alpha_bar= alphas_bar[batched_t, None, None, None]
    x0 = 1. / sqrt_alpha_bar * xt -  jnp.sqrt(1./alpha_bar-1) * noise
    return x0

def x0_to_noise(x0, xt, batched_t, alphas_bar, sqrt_alphas_bar):
    assert batched_t.shape[0] == xt.shape[0] == x0.shape[0] # make sure all has batch dimension
    sqrt_alpha_bar = sqrt_alphas_bar[batched_t, None, None, None]
    alpha_bar= alphas_bar[batched_t, None, None, None]
    noise = (1. / sqrt_alpha_bar * xt - x0) /jnp.sqrt(1./alpha_bar-1)
    return noise

def model_predict(state, x, x0, t, ddpm_param: ddpm_param, use_ema=True):
    if use_ema:
        variables = {'params': state.params_ema}
    else:
        variables = {'params': state.params}
    
    if c.ddpm.self_condition:
        pred = state.apply_fn(variables, jnp.concatenate([x, x0],axis=-1), t)
    else:
        pred = state.apply_fn(variables, x, t)

    if c.ddpm.is_pred_x0: # if the objective is is_pred_x0, pred == x0_pred
        x0_pred = pred
        noise_pred =  x0_to_noise(pred, x, t, ddpm_param.alphas_bar, ddpm_param.sqrt_alphas_bar)
    else:
        noise_pred = pred
        x0_pred = noise_to_x0(pred, x, t, ddpm_param.alphas_bar, ddpm_param.sqrt_alphas_bar)
    
    return x0_pred, noise_pred

def q_sample(x, t, noise, ddpm_param: ddpm_param):
    sqrt_alpha_bar = ddpm_param.sqrt_alphas_bar[t, None, None, None]
    sqrt_1m_alpha_bar = ddpm_param.sqrt_1m_alphas_bar[t,None,None,None]
    x_t = sqrt_alpha_bar * x + sqrt_1m_alpha_bar * noise
    return x_t

def p_loss(rng, state: TrainState, b, loss_fn, ddpm_param: ddpm_param, pmap_axis='batch'):
    rng, t_rng, noise_rng, condition_rng = jax.random.split(rng, 4)

    assert b.dtype in [jnp.float32, jnp.float64]

    B, H, W, C = b.shape
    batched_t = jax.random.randint(t_rng, shape=(B,), dtype = jnp.int32, minval=0, maxval=len(ddpm_param.betas))
    noise = jax.random.normal(noise_rng, b.shape)

    target = b if c.ddpm.is_pred_x0 else noise
    
    x_t = q_sample(b, batched_t, noise, ddpm_param)

    if c.ddpm.self_condition:
        zeros = jnp.zeros_like(x_t)
        
        def estimate_x0(_): # self-conditioning 
            x0, _ = model_predict(state, x_t, zeros, batched_t, ddpm_param, use_ema=False)
            return x0
        
        x0 = jax.lax.cond(
            jax.random.uniform(condition_rng, shape=(1,))[0] < 0.5,
            estimate_x0,
            lambda _ :zeros,
            None,
        )
        
        x_t = jnp.concatenate([x_t, x0], axis=-1)

    def compute_loss(params):
        pred = state.apply_fn({'params':params}, x_t, batched_t)
        loss = loss_fn(flatten(pred),flatten(target))
        loss = jnp.mean(loss, axis=1)
        assert loss.shape == (B,)
        loss = loss * ddpm_param.p2_loss_weight[batched_t]
        return loss.mean()
    
    dynamic_scale = state.dynamic_scale

    if dynamic_scale: # dynamic loss takes care of averaging gradients across replicas
        grad_fn = dynamic_scale.value_and_grad(compute_loss, axis_name=pmap_axis)
        dynamic_scale, is_fin, loss, grads = grad_fn(state.params)
    else: #  Re-use same axis_name as in the call to `pmap(...train_step,axis=...)` in the train function
        grad_fn = jax.value_and_grad(compute_loss)
        loss, grads = grad_fn(state.params)
        grads = jax.lax.pmean(grads, axis_name=pmap_axis)
    
    loss = jax.lax.pmean(loss, axis_name=pmap_axis)
    loss_ema = jax.lax.pmean(compute_loss(state.params_ema), axis_name=pmap_axis)

    metrics = {
        'loss': loss,
        'loss_ema': loss_ema,
    }

    new_state = state.apply_gradients(grads=grads)

    if dynamic_scale: 
        # if is_fin == False the gradients contain Inf/NaNs and optimizer state and # params should be restored (= skip this step).
        new_state=new_state.replace(
            opt_state=jax.tree_map(
                functools.partial(jnp.where, is_fin),
                new_state.opt_state,
                state.opt_state
            ),
            params=jax.tree_map(
                functools.partial(jnp.where, is_fin),
                new_state.params,
                state.params
            ),
            dynamic_scale=dynamic_scale
        )
        metrics['scale'] = dynamic_scale.scale

    return new_state, metrics

train_step = functools.partial(
    p_loss,
    ddpm_param  = ddpm_param, 
    loss_fn     = loss_fn,
    pmap_axis   = 'batch'
)

p_train_step = jax.pmap(train_step, axis_name='batch')

In [ ]:
def get_posterior_mean_variance(img, t, x0, v, ddpm):

    beta = ddpm_param.betas[t,None,None,None] # only needed when t > 0
    alpha = ddpm_param.alphas[t,None,None,None]
    alpha_bar = ddpm_param.alphas_bar[t,None,None,None]
    alpha_bar_last = ddpm_param.alphas_bar[t-1,None,None,None]
    sqrt_alpha_bar_last = ddpm_param.sqrt_alphas_bar[t-1,None,None,None]

    coef_x0 = beta * sqrt_alpha_bar_last / (1. - alpha_bar)
    coef_xt = (1. - alpha_bar_last) * jnp.sqrt(alpha) / ( 1- alpha_bar)        
    posterior_mean = coef_x0 * x0 + coef_xt * img
        
    posterior_variance = beta * (1 - alpha_bar_last) / (1. - alpha_bar)
    posterior_log_variance = jnp.log(jnp.clip(posterior_variance, a_min = 1e-20))

    return posterior_mean, posterior_log_variance


def ddpm_sample_step(state, rng, x, t, x0_last, ddpm_param: ddpm_param):
 
    batched_t = jnp.ones((x.shape[0],), dtype=jnp.int32) * t
    
    if c.ddpm.self_condition:
        x0, v = model_predict(state, x, x0_last, batched_t, ddpm_param, use_ema=True) 
    else:
        x0, v = model_predict(state, x, None, batched_t, ddpm_param, use_ema=True)
    
    x0 = jnp.clip(x0,-1.,1.) # make sure x0 between [-1,1]
    posterior_mean, posterior_log_variance = get_posterior_mean_variance(x, t, x0, v, ddpm_param)
    x = posterior_mean + jnp.exp(0.5 *  posterior_log_variance) * jax.random.normal(rng, x.shape) 
    return x, x0

sample_step = functools.partial(
    ddpm_sample_step, 
    ddpm_param=ddpm_param, 
    self_condition=c.ddpm.self_condition, 
    is_pred_x0=c.ddpm.is_pred_x0
)

p_sample_step = jax.pmap(sample_step, axis_name='batch')

In [ ]:
def copy_params_to_ema(state):
   state = state.replace(params_ema = state.params)
   return state

def apply_ema_decay(state, ema_decay):
    params_ema = jax.tree_map(lambda p_ema, p: p_ema * ema_decay + p * (1. - ema_decay), state.params_ema, state.params)
    state = state.replace(params_ema = params_ema)
    return state

p_apply_ema = jax.pmap(apply_ema_decay, in_axes=(0, None), axis_name='batch')
p_copy_params_to_ema = jax.pmap(copy_params_to_ema, axis_name='batch')


In [ ]:
from pprint import pprint
from pathlib import Path

def to_wandb_config(d, parent_key: str = '', sep: str ='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(to_wandb_config(v, new_key, sep=sep).items())
        else:
            if isinstance(v, Path):
                v = str(v)
            items.append((new_key, v))
    return dict(items)

wandb.init(
    job_type=c.wandb.job_type,
    entity=c.wandb.entity,
    project=c.project,
    config=to_wandb_config(c.dict),
    settings=wandb.Settings(start_method='fork'),  # idk why this is an issue
    dir=c.exp_path,
)

wandb.define_metric("*", step_metric="train/step")

# # Display a project workspace
# %wandb USERNAME/PROJECT
# # Display a single run
# %wandb USERNAME/PROJECT/runs/RUN_ID
# # Display a sweep
# %wandb USERNAME/PROJECT/sweeps/SWEEP_ID
# # Display a report
# %wandb USERNAME/PROJECT/reports/REPORT_ID
# # Specify the height of embedded iframe
# %wandb USERNAME/PROJECT -h 2048

In [ ]:
def get_data_stats():
    def normalize_to_neg_one_to_one(img):
        return img * 2 - 1  
    data_tr = datasets.FashionMNIST(
        root=c.data_dir, 
        download=True, 
        train=True,
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Resize(28),
            transforms.CenterCrop(28),
        ]),
        target_transform = transforms.Compose([
        transforms.Lambda(lambda y: torch.zeros(10, dtype=torch.float).scatter_(dim=0, index=torch.tensor(y), value=1))
    ])
    )
    imgs = torch.stack([img_t for img_t, _ in data_tr], dim=3)
    mean = imgs.view(1,-1).mean(dim=1) 
    std = imgs.view(1,-1).std(dim=1)
    print('Data mean: ', mean, 'Data std: ', std)
    return mean, std
mean, std = get_data_stats()
loader_tr, loader_test = get_fashion_loader(c.data.b_size, c.data_dir, mean=mean, std=std)  # is not an iterator or list
img, l = next(iter(loader_tr))

In [ ]:
import numpy as np
Image.fromarray(np.uint8((img[0, 0]*std+mean).cpu().numpy()*255))

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('1', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))

    return grid
std = float(std.cpu().numpy())
mean = float(mean.cpu().numpy())

In [ ]:
train_metrics = []

for ep in range(c.n_epoch):
    for step, (b, target) in enumerate(loader_tr):
        b = jnp.squeeze(jnp.expand_dims(jnp.array(b.numpy()), axis=(0,-1)), axis=2) # p, B, H, W, C
        rng, *train_step_rng = jax.random.split(rng, num=jax.local_device_count() + 1)
        train_step_rng = jnp.array(train_step_rng)
        
        state, metrics = p_train_step(train_step_rng, state, b)

        if step <= c.ema.update_after_step:
            state = p_copy_params_to_ema(state)

        elif step % c.ema.update_every == 0:
            ema_decay = ema_decay_fn(step)
            state =  p_apply_ema(state, ema_decay)

        if step % c.log_metric_step == 0:

            train_metrics.append(metrics)
            train_metrics = common_utils.get_metrics(train_metrics)
            
            summary = {
                f'train/{k}': v
                for k, v in jax.tree_map(lambda x: x.mean(), train_metrics).items()
            }

            train_metrics = []
            b = ((np.array(b)*std+mean)*255).astype(np.uint8)
            imgs = [wandb.Image(Image.fromarray(b[0, i].reshape(28, 28))) for i in range(9)]
            
            wandb.log({
                    "train/step": step, 
                    'train/sample': imgs,
                    **summary
            })
 
    print('Epoch: ', ep)

In [ ]:
# Image.fromarray(b[0, 0].reshape(28, 28, 1))
b[0, 0].max()